In [10]:
import numpy as np
from tabulate import tabulate
from prettytable import PrettyTable

In [11]:
# defining the grid world
grid = np.zeros((4, 3))
grid[0, 1] = -1  # Penalty
grid[0, 2] = +1   # Reward

#this grid will be modified and copies passed to functions
gridmod=np.zeros((4, 3))
gridmod[0,1]=-1
gridmod[0,2] = +1

#defining parameters
step_cost = -0.04
discount_factor = 0.95
epsilon = 0.0001
p=0.7
p_perpen=0.15
directions = ["up", "left", "down", "right"]
p_values = np.linspace(0.1, 0.9, 9)


In [12]:

opt_grid = np.array([
    ['-', 'none ', 'none '],
    ['-', '-', '-'],
    ['-', 'none ', '-'],
    ['-', '-', '-']
])

In [13]:

def print_table(grid):
    table = PrettyTable()
    for i, row in enumerate(grid):
        formatted_row = []
        for j, value in enumerate(row):
            if (i, j) == (0, 1):   # Highlight cell (0, 1) in red to represent reward
                formatted_row.append(f"\033[91m{value:.4f}\033[0m")
            elif (i, j) == (2, 1):  # Highlight cell (2, 1) in grey to represent wall
                formatted_row.append(f"\033[90m{value:.4f}\033[0m")
            elif (i, j) == (0, 2):   # Highlight cell (0, 2) in green to represent reward
                formatted_row.append(f"\033[92m{value:.4f}\033[0m")
            else:
                formatted_row.append(f"{value:.4f}")
        table.add_row(formatted_row)
    table.hrules = True
    print(table)

print_table(gridmod)

+---------+---------+---------+
| Field 1 | Field 2 | Field 3 |
+---------+---------+---------+
|  0.0000 | -1.0000 |  1.0000 |
+---------+---------+---------+
|  0.0000 |  0.0000 |  0.0000 |
+---------+---------+---------+
|  0.0000 |  0.0000 |  0.0000 |
+---------+---------+---------+
|  0.0000 |  0.0000 |  0.0000 |
+---------+---------+---------+


In [14]:

def move(i, j, grid, direction, p, p_perpen, step_cost, discount_factor):
    # Initialize the value for the direction
    direction_value = 0

    if direction == "up":
        if i-1 == 2 and j == 1:
            direction_value += p * grid[i,j]
        elif i-1 >= 0:
            direction_value += p * grid[i-1, j]
        else:
            direction_value += p * grid[i,j]

        if i == 2 and j+1 == 1:
            direction_value +=  p_perpen * grid[i,j]
        elif j+1 <= 2:
            direction_value += p_perpen * grid[i, j+1]
        else :
            direction_value += p_perpen * grid[i,j]

        if i == 2 and j-1 == 1:
            direction_value +=  p_perpen * grid[i,j]
        elif j-1 >= 0:
            direction_value += p_perpen * grid[i, j-1]
        else :
            direction_value += p_perpen * grid[i,j]

    # Move down
    elif direction == "down":

        if i+1 == 2 and j == 1:
            direction_value += p * grid[i,j]
        elif i+1 <= 3:
            direction_value += p * grid[i+1, j]
        else:
            direction_value += p * grid[i,j]

        if i == 2 and j+1 == 1:
            direction_value +=  p_perpen* grid[i,j]
        elif j+1 <= 2:
            direction_value += p_perpen * grid[i, j+1]
        else :
            direction_value += p_perpen * grid[i,j]

        if i == 2 and j-1 == 1:
            direction_value +=  p_perpen * grid[i,j]
        elif j-1 >= 0:
            direction_value += p_perpen * grid[i, j-1]
        else :
            direction_value += p_perpen * grid[i,j]

    # Move right
    elif direction == "right":
        # Check boundary and walls
        if i == 2 and j+1 == 1:
            direction_value += p * grid[i, j]
        elif j+1 <= 2:
            direction_value += p * grid[i, j+1]
        else:
            direction_value += p * grid[i,j]


        if i+1 == 2 and j == 1:
            direction_value +=  p_perpen * grid[i, j]
        elif i+1 <= 3:
            direction_value += p_perpen * grid[i+1, j]
        else :
            direction_value += p_perpen * grid[i,j]

        if i-1 == 2 and j == 1:
            direction_value +=  p_perpen * grid[i, j]
        elif i-1 >= 0:
            direction_value += p_perpen * grid[i-1, j]
        else :
            direction_value += p_perpen * grid[i,j]


    # Move left
    elif direction == "left":
        # Check boundary and walls
        if i == 2 and j-1 == 1:
            direction_value += p * grid[i, j]
        elif j-1 >= 0:
            direction_value += p * grid[i, j-1]
        else:
            direction_value += p * grid[i,j]

        if i+1 == 2 and j == 1:
            direction_value += p_perpen * grid[i, j]
        elif i+1 <= 3:
            direction_value += p_perpen * grid[i+1, j]
        else :
            direction_value += p_perpen * grid[i,j]

        if i-1 == 2 and j == 1:
            direction_value += p_perpen * grid[i, j]
        elif i-1 >= 0:
            direction_value += p_perpen * grid[i-1, j]
        else :
            direction_value += p_perpen * grid[i,j]

    # Calculate the final value considering step cost and discount factor
    direction_val = step_cost + discount_factor * direction_value
    return direction_val

move(3, 2, grid, "up", 0.7, 0.15, -0.04, 0.95)

-0.04

In [15]:

def mdp(grid, gridmod,p,p_perpen, opt_grid):
    for i in range(0, 4):
        for j in range(0, 3):
            if (i == 2 and j == 1) or (i == 0 and j == 1) or (i == 0 and j == 2):
                #wall or final states
                continue
            else:
                max_value = float("-inf")
                for direction in directions:
                    value = move(i, j, grid, direction, p, p_perpen, step_cost, discount_factor)
                    max_value = max(max_value, value)
                    if(max_value == value):
                        opt_grid[i,j] = direction
                gridmod[i, j] = max_value 
    return gridmod

In [16]:
def valueIter(grid, gridmod, p, p_perpen, epsilon, opt_grid, flag):
    iterations = 0
    while True:
        grid = np.copy(gridmod)
        gridmod = mdp(grid, gridmod,p,p_perpen, opt_grid)
        
        if flag == 1:
            print_table(gridmod)
            iterations += 1
            print("\n")
        if np.max(np.abs(grid - gridmod)) < epsilon:
            grid = np.copy(gridmod)
            break
    return gridmod, iterations

final_grid, iterations = valueIter(grid, gridmod, p, p_perpen, epsilon, opt_grid, 1)
print(f"Grid world after {iterations} iterations (convergence reached):\n") #, tabulate(np.around(final_grid, decimals=4)))
print_table(final_grid)

+---------+---------+---------+
| Field 1 | Field 2 | Field 3 |
+---------+---------+---------+
| -0.0400 | -1.0000 |  1.0000 |
+---------+---------+---------+
| -0.0400 | -0.0400 |  0.6250 |
+---------+---------+---------+
| -0.0400 |  0.0000 | -0.0400 |
+---------+---------+---------+
| -0.0400 | -0.0400 | -0.0400 |
+---------+---------+---------+


+---------+---------+---------+
| Field 1 | Field 2 | Field 3 |
+---------+---------+---------+
| -0.0780 | -1.0000 |  1.0000 |
+---------+---------+---------+
| -0.0780 |  0.2274 |  0.7084 |
+---------+---------+---------+
| -0.0780 |  0.0000 |  0.3642 |
+---------+---------+---------+
| -0.0780 | -0.0780 | -0.0780 |
+---------+---------+---------+


+---------+---------+---------+
| Field 1 | Field 2 | Field 3 |
+---------+---------+---------+
| -0.1141 | -1.0000 |  1.0000 |
+---------+---------+---------+
|  0.0890 |  0.3210 |  0.7583 |
+---------+---------+---------+
| -0.1141 |  0.0000 |  0.5349 |
+---------+---------+---------+
| -0

In [17]:
print(opt_grid)

[['down' 'none ' 'none ']
 ['right' 'right' 'up']
 ['down' 'none ' 'up']
 ['right' 'right' 'up']]


In [18]:

for p_new in p_values:
  opt_grid = np.array([
    ['-', 'none ', 'none '],
    ['-', '-', '-'],
    ['-', 'none ', '-'],
    ['-', '-', '-']
])
  p_perpen = (1-p_new)/2
  valueIter(grid, gridmod, p_new, p_perpen, epsilon, opt_grid, 2)
  print(f"Final Grid for p = {p_new} is:") 
  print(opt_grid)

Final Grid for p = 0.1 is:
[['left' 'none ' 'none ']
 ['up' 'down' 'right']
 ['right' 'none ' 'right']
 ['up' 'down' 'right']]
Final Grid for p = 0.2 is:
[['left' 'none ' 'none ']
 ['up' 'down' 'right']
 ['right' 'none ' 'right']
 ['up' 'down' 'right']]
Final Grid for p = 0.30000000000000004 is:
[['left' 'none ' 'none ']
 ['down' 'down' 'right']
 ['right' 'none ' 'up']
 ['down' 'right' 'right']]
Final Grid for p = 0.4 is:
[['left' 'none ' 'none ']
 ['down' 'down' 'right']
 ['right' 'none ' 'up']
 ['down' 'right' 'right']]
Final Grid for p = 0.5 is:
[['left' 'none ' 'none ']
 ['right' 'down' 'up']
 ['down' 'none ' 'up']
 ['right' 'right' 'up']]
Final Grid for p = 0.6 is:
[['left' 'none ' 'none ']
 ['right' 'down' 'up']
 ['down' 'none ' 'up']
 ['right' 'right' 'up']]
Final Grid for p = 0.7000000000000001 is:
[['down' 'none ' 'none ']
 ['right' 'right' 'up']
 ['down' 'none ' 'up']
 ['right' 'right' 'up']]
Final Grid for p = 0.8 is:
[['down' 'none ' 'none ']
 ['right' 'right' 'up']
 ['down

As we can see, the optimal moves change as we change the value of p, which is the probability of a move being executed as chosen. at p = 0.1 for example in the top left corner (0, 0), the left move is most optimal whereas for p=0.9 down is most optimal. This happens because we can reliably choose a move that maximizes reward for larger values of p, whereas with smaller values of p, there is a much greater element of randomness. 

Even if we choose a direction which maximizes reward, there is a relatively high probability of not going in that direction, so it may be a better strategy to choose a deterministically non-optimal choice which can maximize utility by not going in the selected direction. This becomes unviable with higher p.